In [129]:
import pandas as pd
import numpy as np

**Часть, отвечающая за тип запроса**

Загрузим обучающие и тестовые данные

In [130]:
data_train = pd.read_csv("/Users/natala/Desktop/Хакатончик-тончик/Goptar/KDDTrain+.csv")
data_test = pd.read_csv("/Users/natala/Desktop/Хакатончик-тончик/Goptar/KDDTest+.csv")

In [131]:
data_train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [132]:
data_test.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,private,REJ,0,0,0,0,0,0,...,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,anomaly
1,0,tcp,private,REJ,0,0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,anomaly
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,anomaly
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,anomaly


X_train/X_test - матрица признаков;

y_train/y_test -  вектор значений;

In [133]:
y_train = data_train['class']
X_train = data_train.drop('class', axis=1)

In [134]:
y_test = data_test['class']
X_test = data_test.drop('class', axis=1)

In [135]:
y_test

0        anomaly
1        anomaly
2         normal
3        anomaly
4        anomaly
          ...   
22539     normal
22540     normal
22541    anomaly
22542     normal
22543    anomaly
Name: class, Length: 22544, dtype: object

**Обработка данных**

Преобразуем вектора значения в булевый тип (normal = 1, anomaly = 0)

In [136]:
y_train = np.where(y_train == "normal", 1, 0)
y_test = np.where(y_test == "normal", 1, 0)

Преобразуем поля protocol_type, service, flag d X_train и X_test в числовой тип

In [137]:
from sklearn import preprocessing #импорт необходимого модуля 

le_protocol = preprocessing.LabelEncoder() #получение меток
le_protocol.fit(X_train["protocol_type"]) #кодирование меток
X_train["protocol_type"] = le_protocol.transform(X_train["protocol_type"]) #преобразование меток
X_test["protocol_type"] = le_protocol.transform(X_test["protocol_type"])

le_service = preprocessing.LabelEncoder()
le_service.fit(X_train["service"])
X_train["service"] = le_service.transform(X_train["service"])
X_test["service"] = le_service.transform(X_test["service"])

le_flag = preprocessing.LabelEncoder() 
le_flag.fit(X_train["flag"]) 
X_train["flag"] = le_flag.transform(X_train["flag"]) 
X_test["flag"] = le_flag.transform(X_test["flag"])


**Модель**

Создадим и обучим модель 

In [140]:
!pip install catboost

     |████████████████████████████████| 11.1 MB 430 kB/s eta 0:00:01    |██████████████████████████      | 9.0 MB 110 kB/s eta 0:00:19
     |████████████████████████████████| 12.9 MB 757 kB/s eta 0:00:01    |██████████████████████          | 8.8 MB 617 kB/s eta 0:00:07     |███████████████████████████▏    | 10.9 MB 289 kB/s eta 0:00:07
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=4368b0ee80a0b3c842cff284e17f7a9d0c061e1f9448e9046e89ead83397322b
  Stored in directory: /Users/natala/Library/Caches/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [141]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from collections import Counter
from tqdm import tqdm
from sklearn.cluster import KMeans

In [142]:
cat = CatBoostClassifier(iterations = 2000, learning_rate = 0.05, depth = 3, loss_function = 'CrossEntropy', eval_metric='Accuracy',
                         random_state = 53, use_best_model=True)

train_data = Pool(
    data = X_train,
    label = y_train
)

valid_data = Pool(
    data = X_test,
    label = y_test
)

In [143]:
cat.fit(train_data, eval_set = valid_data)

0:	learn: 0.9499417	test: 0.8027413	best: 0.8027413 (0)	total: 146ms	remaining: 4m 52s
1:	learn: 0.9568082	test: 0.8048261	best: 0.8048261 (1)	total: 216ms	remaining: 3m 36s
2:	learn: 0.9565462	test: 0.8148953	best: 0.8148953 (2)	total: 269ms	remaining: 2m 59s
3:	learn: 0.9668739	test: 0.8040277	best: 0.8148953 (2)	total: 325ms	remaining: 2m 42s
4:	learn: 0.9659213	test: 0.7912083	best: 0.8148953 (2)	total: 394ms	remaining: 2m 37s
5:	learn: 0.9612536	test: 0.8062899	best: 0.8148953 (2)	total: 450ms	remaining: 2m 29s
6:	learn: 0.9613012	test: 0.8024752	best: 0.8148953 (2)	total: 499ms	remaining: 2m 22s
7:	learn: 0.9631111	test: 0.7950231	best: 0.8148953 (2)	total: 553ms	remaining: 2m 17s
8:	learn: 0.9631350	test: 0.7959989	best: 0.8148953 (2)	total: 609ms	remaining: 2m 14s
9:	learn: 0.9684377	test: 0.7963094	best: 0.8148953 (2)	total: 791ms	remaining: 2m 37s
10:	learn: 0.9685250	test: 0.7975515	best: 0.8148953 (2)	total: 843ms	remaining: 2m 32s
11:	learn: 0.9698031	test: 0.7949787	best:

94:	learn: 0.9920459	test: 0.7810504	best: 0.8148953 (2)	total: 6.19s	remaining: 2m 4s
95:	learn: 0.9921174	test: 0.7855305	best: 0.8148953 (2)	total: 6.24s	remaining: 2m 3s
96:	learn: 0.9921015	test: 0.7809617	best: 0.8148953 (2)	total: 6.32s	remaining: 2m 4s
97:	learn: 0.9920935	test: 0.7808730	best: 0.8148953 (2)	total: 6.37s	remaining: 2m 3s
98:	learn: 0.9922444	test: 0.7809173	best: 0.8148953 (2)	total: 6.42s	remaining: 2m 3s
99:	learn: 0.9921650	test: 0.7807842	best: 0.8148953 (2)	total: 6.47s	remaining: 2m 2s
100:	learn: 0.9922364	test: 0.7853974	best: 0.8148953 (2)	total: 6.52s	remaining: 2m 2s
101:	learn: 0.9922206	test: 0.7853974	best: 0.8148953 (2)	total: 6.62s	remaining: 2m 3s
102:	learn: 0.9921967	test: 0.7853531	best: 0.8148953 (2)	total: 6.73s	remaining: 2m 3s
103:	learn: 0.9922047	test: 0.7853087	best: 0.8148953 (2)	total: 6.8s	remaining: 2m 3s
104:	learn: 0.9926333	test: 0.7852644	best: 0.8148953 (2)	total: 6.83s	remaining: 2m 3s
105:	learn: 0.9927286	test: 0.7851313	b

188:	learn: 0.9958245	test: 0.7865951	best: 0.8148953 (2)	total: 11.4s	remaining: 1m 49s
189:	learn: 0.9958324	test: 0.7868612	best: 0.8148953 (2)	total: 11.5s	remaining: 1m 49s
190:	learn: 0.9958324	test: 0.7866838	best: 0.8148953 (2)	total: 11.5s	remaining: 1m 49s
191:	learn: 0.9958324	test: 0.7869056	best: 0.8148953 (2)	total: 11.6s	remaining: 1m 49s
192:	learn: 0.9958721	test: 0.7865507	best: 0.8148953 (2)	total: 11.6s	remaining: 1m 48s
193:	learn: 0.9958721	test: 0.7865064	best: 0.8148953 (2)	total: 11.8s	remaining: 1m 49s
194:	learn: 0.9958721	test: 0.7864177	best: 0.8148953 (2)	total: 11.9s	remaining: 1m 49s
195:	learn: 0.9958563	test: 0.7863733	best: 0.8148953 (2)	total: 11.9s	remaining: 1m 49s
196:	learn: 0.9958959	test: 0.7863733	best: 0.8148953 (2)	total: 12s	remaining: 1m 49s
197:	learn: 0.9958880	test: 0.7865064	best: 0.8148953 (2)	total: 12.1s	remaining: 1m 49s
198:	learn: 0.9958801	test: 0.7866395	best: 0.8148953 (2)	total: 12.2s	remaining: 1m 49s
199:	learn: 0.9959039	t

283:	learn: 0.9967930	test: 0.7863290	best: 0.8148953 (2)	total: 17.4s	remaining: 1m 45s
284:	learn: 0.9968009	test: 0.7870830	best: 0.8148953 (2)	total: 17.5s	remaining: 1m 45s
285:	learn: 0.9967930	test: 0.7868612	best: 0.8148953 (2)	total: 17.5s	remaining: 1m 45s
286:	learn: 0.9967930	test: 0.7868612	best: 0.8148953 (2)	total: 17.6s	remaining: 1m 45s
287:	learn: 0.9968009	test: 0.7867725	best: 0.8148953 (2)	total: 17.7s	remaining: 1m 44s
288:	learn: 0.9968009	test: 0.7868612	best: 0.8148953 (2)	total: 17.7s	remaining: 1m 44s
289:	learn: 0.9968009	test: 0.7869943	best: 0.8148953 (2)	total: 17.8s	remaining: 1m 44s
290:	learn: 0.9968009	test: 0.7869056	best: 0.8148953 (2)	total: 17.8s	remaining: 1m 44s
291:	learn: 0.9968168	test: 0.7873492	best: 0.8148953 (2)	total: 17.8s	remaining: 1m 44s
292:	learn: 0.9968247	test: 0.7873048	best: 0.8148953 (2)	total: 17.9s	remaining: 1m 44s
293:	learn: 0.9968327	test: 0.7869500	best: 0.8148953 (2)	total: 17.9s	remaining: 1m 43s
294:	learn: 0.9968565

380:	learn: 0.9971264	test: 0.7869943	best: 0.8148953 (2)	total: 22.6s	remaining: 1m 36s
381:	learn: 0.9971264	test: 0.7869943	best: 0.8148953 (2)	total: 22.7s	remaining: 1m 36s
382:	learn: 0.9971343	test: 0.7869943	best: 0.8148953 (2)	total: 22.8s	remaining: 1m 36s
383:	learn: 0.9971422	test: 0.7870387	best: 0.8148953 (2)	total: 22.8s	remaining: 1m 35s
384:	learn: 0.9971422	test: 0.7869943	best: 0.8148953 (2)	total: 22.9s	remaining: 1m 35s
385:	learn: 0.9970787	test: 0.7867282	best: 0.8148953 (2)	total: 22.9s	remaining: 1m 35s
386:	learn: 0.9970787	test: 0.7867282	best: 0.8148953 (2)	total: 22.9s	remaining: 1m 35s
387:	learn: 0.9970787	test: 0.7867282	best: 0.8148953 (2)	total: 23s	remaining: 1m 35s
388:	learn: 0.9970708	test: 0.7866838	best: 0.8148953 (2)	total: 23s	remaining: 1m 35s
389:	learn: 0.9970787	test: 0.7867725	best: 0.8148953 (2)	total: 23s	remaining: 1m 35s
390:	learn: 0.9970708	test: 0.7866838	best: 0.8148953 (2)	total: 23.1s	remaining: 1m 34s
391:	learn: 0.9970708	test:

...

1539:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 1s	remaining: 18.4s
1540:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 1s	remaining: 18.4s
1541:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 1s	remaining: 18.3s
1542:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 1s	remaining: 18.3s
1543:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 1s	remaining: 18.2s
1544:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 1s	remaining: 18.2s
1545:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 1s	remaining: 18.2s
1546:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 1s	remaining: 18.1s
1547:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 2s	remaining: 18.1s
1548:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 2s	remaining: 18.1s
1549:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 2s	remaining: 18s
1550:	learn: 0.9971899	

1637:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 4s	remaining: 14.2s
1638:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 4s	remaining: 14.2s
1639:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 4s	remaining: 14.2s
1640:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 4s	remaining: 14.1s
1641:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 4s	remaining: 14.1s
1642:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 4s	remaining: 14s
1643:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 4s	remaining: 14s
1644:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 4s	remaining: 13.9s
1645:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 4s	remaining: 13.9s
1646:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 4s	remaining: 13.9s
1647:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 4s	remaining: 13.8s
1648:	learn: 0.9971899	te

1736:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 8s	remaining: 10.3s
1737:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 8s	remaining: 10.3s
1738:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 8s	remaining: 10.2s
1739:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 8s	remaining: 10.2s
1740:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 8s	remaining: 10.1s
1741:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 8s	remaining: 10.1s
1742:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 8s	remaining: 10.1s
1743:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 8s	remaining: 10s
1744:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 8s	remaining: 9.98s
1745:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 8s	remaining: 9.94s
1746:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 8s	remaining: 9.9s
1747:	learn: 0.9971899	t

1834:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 11s	remaining: 6.39s
1835:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 11s	remaining: 6.35s
1836:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 11s	remaining: 6.31s
1837:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 11s	remaining: 6.27s
1838:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 11s	remaining: 6.23s
1839:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 11s	remaining: 6.19s
1840:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 11s	remaining: 6.16s
1841:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 11s	remaining: 6.12s
1842:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 11s	remaining: 6.08s
1843:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 11s	remaining: 6.04s
1844:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 11s	remaining: 6s
1845:	learn: 

1927:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 14s	remaining: 2.78s
1928:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 14s	remaining: 2.74s
1929:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 14s	remaining: 2.7s
1930:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 14s	remaining: 2.66s
1931:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 14s	remaining: 2.63s
1932:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 14s	remaining: 2.59s
1933:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 14s	remaining: 2.55s
1934:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 14s	remaining: 2.51s
1935:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 14s	remaining: 2.48s
1936:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 14s	remaining: 2.44s
1937:	learn: 0.9971899	test: 0.7869943	best: 0.8148953 (2)	total: 1m 15s	remaining: 2.4s
1938:	learn:

Конец части, определяющий тип запроса

**Часть, определяющая вид атаки**

In [ ]:
Загрузим обучающие и тестовые данные

In [160]:
train_multi = pd.read_csv("/Users/natala/Desktop/Хакатончик-тончик/Goptar/KDDTrain+_meted.csv")
test_multi = pd.read_csv("/Users/natala/Desktop/Хакатончик-тончик/Goptar/KDDTest+_meted.csv")

In [161]:
test_multi.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,Unnamed: 42
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal,21
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint,15
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan,11


In [162]:
train_multi.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class,Unnamed: 42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


**Подготовка данных**

In [163]:
train_multi = train_multi.drop("Unnamed: 42", axis=1)
test_multi = test_multi.drop("Unnamed: 42", axis=1)

In [164]:
test_unknown = ['apache2', 'httptunnel', 'mailbomb', 'mscan', 'named', 'processtable', 'ps', 'saint',
'sendmail', 'snmpgetattack', 'snmpguess', 'sqlattack', 'udpstorm', 'worm', 'xlock', 'xsnoop', 'xterm']

train_multi = train_multi[train_multi['class'] != "normal"]
test_multi = test_multi[test_multi['class'] != "normal"]

for name in test_unknown:
  test_multi = test_multi[test_multi['class'] != name]

In [165]:
y_train_multi = train_multi["class"]
X_train_multi = train_multi.drop("class", axis=1)

y_test_multi = test_multi["class"]
X_test_multi = test_multi.drop("class", axis=1)

In [166]:
le_protocol_multi = preprocessing.LabelEncoder()
le_protocol_multi.fit(X_train_multi["protocol_type"])
X_train_multi["protocol_type"] = le_protocol_multi.transform(X_train_multi["protocol_type"])
X_test_multi["protocol_type"] = le_protocol_multi.transform(X_test_multi["protocol_type"])

le_service_multi = preprocessing.LabelEncoder()
le_service_multi.fit(X_train_multi["service"])
X_train_multi["service"] = le_service_multi.transform(X_train_multi["service"])
X_test_multi["service"] = le_service_multi.transform(X_test_multi["service"])

le_flag_multi = preprocessing.LabelEncoder()
le_flag_multi.fit(X_train_multi["flag"])
X_train_multi["flag"] = le_flag_multi.transform(X_train_multi["flag"])
X_test_multi["flag"] = le_flag_multi.transform(X_test_multi["flag"])

In [167]:
le_label_multi = preprocessing.LabelEncoder()
le_label_multi.fit(y_train_multi)
train_multi_y = le_label_multi.transform(y_train_multi)
test_multi_y = le_label_multi.transform(y_test_multi)

In [168]:
cat_multi = CatBoostClassifier(iterations = 1000, learning_rate = 0.1, depth = 5, loss_function = 'MultiClass', eval_metric='Accuracy',
                         random_state = 53, use_best_model=True)

train_data_multi = Pool(
    data = X_train_multi,
    label = y_train_multi
)

valid_data_multi = Pool(
    data = X_test_multi,
    label = y_test_multi
)

In [169]:
cat_multi.fit(train_data_multi, eval_set = valid_data_multi)

0:	learn: 0.9330548	test: 0.7319168	best: 0.7319168 (0)	total: 256ms	remaining: 4m 15s
1:	learn: 0.9481153	test: 0.7331278	best: 0.7331278 (1)	total: 471ms	remaining: 3m 55s
2:	learn: 0.9568821	test: 0.7335682	best: 0.7335682 (2)	total: 736ms	remaining: 4m 4s
3:	learn: 0.9658707	test: 0.7344490	best: 0.7344490 (3)	total: 1.08s	remaining: 4m 28s
4:	learn: 0.9748081	test: 0.7403941	best: 0.7403941 (4)	total: 1.32s	remaining: 4m 23s
5:	learn: 0.9778953	test: 0.7406143	best: 0.7406143 (5)	total: 1.55s	remaining: 4m 17s
6:	learn: 0.9787310	test: 0.7407244	best: 0.7407244 (6)	total: 1.85s	remaining: 4m 22s
7:	learn: 0.9802831	test: 0.7407244	best: 0.7407244 (6)	total: 2.14s	remaining: 4m 25s
8:	learn: 0.9813747	test: 0.7408345	best: 0.7408345 (8)	total: 2.46s	remaining: 4m 30s
9:	learn: 0.9829609	test: 0.7430364	best: 0.7430364 (9)	total: 2.66s	remaining: 4m 23s
10:	learn: 0.9840525	test: 0.7431465	best: 0.7431465 (10)	total: 2.92s	remaining: 4m 22s
11:	learn: 0.9895617	test: 0.7441374	best:

93:	learn: 0.9972199	test: 0.7490917	best: 0.7493119 (86)	total: 20.7s	remaining: 3m 19s
94:	learn: 0.9972199	test: 0.7490917	best: 0.7493119 (86)	total: 20.9s	remaining: 3m 19s
95:	learn: 0.9972199	test: 0.7490917	best: 0.7493119 (86)	total: 21.1s	remaining: 3m 19s
96:	learn: 0.9972369	test: 0.7490917	best: 0.7493119 (86)	total: 21.4s	remaining: 3m 18s
97:	learn: 0.9972540	test: 0.7490917	best: 0.7493119 (86)	total: 21.6s	remaining: 3m 18s
98:	learn: 0.9972540	test: 0.7492018	best: 0.7493119 (86)	total: 21.8s	remaining: 3m 18s
99:	learn: 0.9974757	test: 0.7492018	best: 0.7493119 (86)	total: 22s	remaining: 3m 17s
100:	learn: 0.9974586	test: 0.7492018	best: 0.7493119 (86)	total: 22.2s	remaining: 3m 17s
101:	learn: 0.9975098	test: 0.7492018	best: 0.7493119 (86)	total: 22.4s	remaining: 3m 17s
102:	learn: 0.9973734	test: 0.7492018	best: 0.7493119 (86)	total: 22.6s	remaining: 3m 16s
103:	learn: 0.9973904	test: 0.7492018	best: 0.7493119 (86)	total: 22.8s	remaining: 3m 16s
104:	learn: 0.99744

185:	learn: 0.9986867	test: 0.7536056	best: 0.7536056 (168)	total: 40.6s	remaining: 2m 57s
186:	learn: 0.9986867	test: 0.7536056	best: 0.7536056 (168)	total: 40.8s	remaining: 2m 57s
187:	learn: 0.9986867	test: 0.7536056	best: 0.7536056 (168)	total: 41.1s	remaining: 2m 57s
188:	learn: 0.9986867	test: 0.7545965	best: 0.7545965 (188)	total: 41.3s	remaining: 2m 57s
189:	learn: 0.9986867	test: 0.7541561	best: 0.7545965 (188)	total: 41.5s	remaining: 2m 56s
190:	learn: 0.9986526	test: 0.7537157	best: 0.7545965 (188)	total: 41.7s	remaining: 2m 56s
191:	learn: 0.9987378	test: 0.7537157	best: 0.7545965 (188)	total: 42s	remaining: 2m 56s
192:	learn: 0.9987720	test: 0.7545965	best: 0.7545965 (188)	total: 42.2s	remaining: 2m 56s
193:	learn: 0.9987720	test: 0.7550369	best: 0.7550369 (193)	total: 42.4s	remaining: 2m 56s
194:	learn: 0.9987720	test: 0.7545965	best: 0.7550369 (193)	total: 42.6s	remaining: 2m 56s
195:	learn: 0.9987720	test: 0.7550369	best: 0.7550369 (193)	total: 42.9s	remaining: 2m 55s
1

276:	learn: 0.9990449	test: 0.7614224	best: 0.7614224 (267)	total: 1m	remaining: 2m 38s
277:	learn: 0.9990619	test: 0.7613123	best: 0.7614224 (267)	total: 1m	remaining: 2m 38s
278:	learn: 0.9990619	test: 0.7613123	best: 0.7614224 (267)	total: 1m 1s	remaining: 2m 37s
279:	learn: 0.9990790	test: 0.7613123	best: 0.7614224 (267)	total: 1m 1s	remaining: 2m 37s
280:	learn: 0.9990790	test: 0.7613123	best: 0.7614224 (267)	total: 1m 1s	remaining: 2m 37s
281:	learn: 0.9990790	test: 0.7613123	best: 0.7614224 (267)	total: 1m 1s	remaining: 2m 37s
282:	learn: 0.9990960	test: 0.7613123	best: 0.7614224 (267)	total: 1m 1s	remaining: 2m 37s
283:	learn: 0.9990960	test: 0.7613123	best: 0.7614224 (267)	total: 1m 2s	remaining: 2m 36s
284:	learn: 0.9990790	test: 0.7613123	best: 0.7614224 (267)	total: 1m 2s	remaining: 2m 36s
285:	learn: 0.9990790	test: 0.7613123	best: 0.7614224 (267)	total: 1m 2s	remaining: 2m 36s
286:	learn: 0.9990960	test: 0.7613123	best: 0.7614224 (267)	total: 1m 2s	remaining: 2m 36s
287:	

366:	learn: 0.9993689	test: 0.7642849	best: 0.7642849 (366)	total: 1m 21s	remaining: 2m 20s
367:	learn: 0.9993689	test: 0.7641748	best: 0.7642849 (366)	total: 1m 21s	remaining: 2m 20s
368:	learn: 0.9993860	test: 0.7640647	best: 0.7642849 (366)	total: 1m 21s	remaining: 2m 19s
369:	learn: 0.9993689	test: 0.7640647	best: 0.7642849 (366)	total: 1m 22s	remaining: 2m 19s
370:	learn: 0.9993689	test: 0.7641748	best: 0.7642849 (366)	total: 1m 22s	remaining: 2m 19s
371:	learn: 0.9993689	test: 0.7641748	best: 0.7642849 (366)	total: 1m 22s	remaining: 2m 19s
372:	learn: 0.9993860	test: 0.7641748	best: 0.7642849 (366)	total: 1m 22s	remaining: 2m 18s
373:	learn: 0.9993689	test: 0.7642849	best: 0.7642849 (366)	total: 1m 22s	remaining: 2m 18s
374:	learn: 0.9993689	test: 0.7641748	best: 0.7642849 (366)	total: 1m 23s	remaining: 2m 18s
375:	learn: 0.9993689	test: 0.7641748	best: 0.7642849 (366)	total: 1m 23s	remaining: 2m 18s
376:	learn: 0.9993860	test: 0.7641748	best: 0.7642849 (366)	total: 1m 23s	remain

...

546:	learn: 0.9995907	test: 0.7654960	best: 0.7657162 (449)	total: 1m 57s	remaining: 1m 37s
547:	learn: 0.9995907	test: 0.7654960	best: 0.7657162 (449)	total: 1m 57s	remaining: 1m 37s
548:	learn: 0.9995907	test: 0.7654960	best: 0.7657162 (449)	total: 1m 57s	remaining: 1m 36s
549:	learn: 0.9995907	test: 0.7654960	best: 0.7657162 (449)	total: 1m 58s	remaining: 1m 36s
550:	learn: 0.9996077	test: 0.7654960	best: 0.7657162 (449)	total: 1m 58s	remaining: 1m 36s
551:	learn: 0.9996077	test: 0.7654960	best: 0.7657162 (449)	total: 1m 58s	remaining: 1m 36s
552:	learn: 0.9996077	test: 0.7654960	best: 0.7657162 (449)	total: 1m 58s	remaining: 1m 35s
553:	learn: 0.9996077	test: 0.7654960	best: 0.7657162 (449)	total: 1m 58s	remaining: 1m 35s
554:	learn: 0.9996077	test: 0.7654960	best: 0.7657162 (449)	total: 1m 58s	remaining: 1m 35s
555:	learn: 0.9996077	test: 0.7654960	best: 0.7657162 (449)	total: 1m 59s	remaining: 1m 35s
556:	learn: 0.9996077	test: 0.7654960	best: 0.7657162 (449)	total: 1m 59s	remain

637:	learn: 0.9996759	test: 0.7656061	best: 0.7665969 (618)	total: 2m 14s	remaining: 1m 16s
638:	learn: 0.9996759	test: 0.7656061	best: 0.7665969 (618)	total: 2m 14s	remaining: 1m 15s
639:	learn: 0.9996759	test: 0.7657162	best: 0.7665969 (618)	total: 2m 14s	remaining: 1m 15s
640:	learn: 0.9996759	test: 0.7657162	best: 0.7665969 (618)	total: 2m 14s	remaining: 1m 15s
641:	learn: 0.9996589	test: 0.7657162	best: 0.7665969 (618)	total: 2m 14s	remaining: 1m 15s
642:	learn: 0.9996589	test: 0.7657162	best: 0.7665969 (618)	total: 2m 15s	remaining: 1m 15s
643:	learn: 0.9996589	test: 0.7657162	best: 0.7665969 (618)	total: 2m 15s	remaining: 1m 14s
644:	learn: 0.9996589	test: 0.7656061	best: 0.7665969 (618)	total: 2m 15s	remaining: 1m 14s
645:	learn: 0.9996759	test: 0.7656061	best: 0.7665969 (618)	total: 2m 15s	remaining: 1m 14s
646:	learn: 0.9996759	test: 0.7656061	best: 0.7665969 (618)	total: 2m 15s	remaining: 1m 14s
647:	learn: 0.9996759	test: 0.7656061	best: 0.7665969 (618)	total: 2m 16s	remain

727:	learn: 0.9997271	test: 0.7670373	best: 0.7671474 (714)	total: 2m 31s	remaining: 56.6s
728:	learn: 0.9997271	test: 0.7670373	best: 0.7671474 (714)	total: 2m 31s	remaining: 56.3s
729:	learn: 0.9997271	test: 0.7670373	best: 0.7671474 (714)	total: 2m 31s	remaining: 56.1s
730:	learn: 0.9997271	test: 0.7670373	best: 0.7671474 (714)	total: 2m 31s	remaining: 55.9s
731:	learn: 0.9997271	test: 0.7670373	best: 0.7671474 (714)	total: 2m 32s	remaining: 55.7s
732:	learn: 0.9997271	test: 0.7668171	best: 0.7671474 (714)	total: 2m 32s	remaining: 55.5s
733:	learn: 0.9997271	test: 0.7668171	best: 0.7671474 (714)	total: 2m 32s	remaining: 55.3s
734:	learn: 0.9997271	test: 0.7669272	best: 0.7671474 (714)	total: 2m 32s	remaining: 55.1s
735:	learn: 0.9997271	test: 0.7670373	best: 0.7671474 (714)	total: 2m 32s	remaining: 54.9s
736:	learn: 0.9997271	test: 0.7670373	best: 0.7671474 (714)	total: 2m 33s	remaining: 54.6s
737:	learn: 0.9997100	test: 0.7671474	best: 0.7671474 (714)	total: 2m 33s	remaining: 54.4s

818:	learn: 0.9997612	test: 0.7685787	best: 0.7686888 (801)	total: 2m 50s	remaining: 37.7s
819:	learn: 0.9997612	test: 0.7685787	best: 0.7686888 (801)	total: 2m 50s	remaining: 37.5s
820:	learn: 0.9997612	test: 0.7685787	best: 0.7686888 (801)	total: 2m 50s	remaining: 37.2s
821:	learn: 0.9997612	test: 0.7684686	best: 0.7686888 (801)	total: 2m 51s	remaining: 37s
822:	learn: 0.9997612	test: 0.7685787	best: 0.7686888 (801)	total: 2m 51s	remaining: 36.8s
823:	learn: 0.9997612	test: 0.7685787	best: 0.7686888 (801)	total: 2m 51s	remaining: 36.6s
824:	learn: 0.9997612	test: 0.7685787	best: 0.7686888 (801)	total: 2m 51s	remaining: 36.4s
825:	learn: 0.9997612	test: 0.7685787	best: 0.7686888 (801)	total: 2m 51s	remaining: 36.2s
826:	learn: 0.9997612	test: 0.7685787	best: 0.7686888 (801)	total: 2m 52s	remaining: 36s
827:	learn: 0.9997612	test: 0.7685787	best: 0.7686888 (801)	total: 2m 52s	remaining: 35.8s
828:	learn: 0.9997612	test: 0.7685787	best: 0.7686888 (801)	total: 2m 52s	remaining: 35.6s
829

910:	learn: 0.9997783	test: 0.7714412	best: 0.7715512 (862)	total: 3m 8s	remaining: 18.4s
911:	learn: 0.9997783	test: 0.7714412	best: 0.7715512 (862)	total: 3m 9s	remaining: 18.2s
912:	learn: 0.9997783	test: 0.7715512	best: 0.7715512 (862)	total: 3m 9s	remaining: 18s
913:	learn: 0.9997783	test: 0.7715512	best: 0.7715512 (862)	total: 3m 9s	remaining: 17.8s
914:	learn: 0.9997783	test: 0.7715512	best: 0.7715512 (862)	total: 3m 9s	remaining: 17.6s
915:	learn: 0.9997783	test: 0.7714412	best: 0.7715512 (862)	total: 3m 9s	remaining: 17.4s
916:	learn: 0.9997783	test: 0.7714412	best: 0.7715512 (862)	total: 3m 9s	remaining: 17.2s
917:	learn: 0.9997783	test: 0.7714412	best: 0.7715512 (862)	total: 3m 10s	remaining: 17s
918:	learn: 0.9997783	test: 0.7715512	best: 0.7715512 (862)	total: 3m 10s	remaining: 16.8s
919:	learn: 0.9997783	test: 0.7714412	best: 0.7715512 (862)	total: 3m 10s	remaining: 16.6s
920:	learn: 0.9997783	test: 0.7716613	best: 0.7716613 (920)	total: 3m 10s	remaining: 16.4s
921:	learn